In [ ]:
!pip install pymongo
import pandas as pd
import string, json, sys, nltk, time, ssl
import numpy as np
from pymongo import MongoClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from multiprocessing import Pool

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download("vader_lexicon")

start_time = time.time()

client = MongoClient("mongodb://192.168.1.27:27017/?readPreference=primary&appname=MongoDB%20Compass&ssl=false")
mydb = client["TwitterSentimentAnalysis"]
mycol = mydb["Covid19"]


noRetweet = mycol.distinct("full_text", {'lang': 'en', 'retweeted_status': None})
Retweet = mycol.distinct("retweeted_status.full_text", {'lang': 'en', 'retweeted_status': {'$ne': None}, 'retweeted_status.lang':'en'})

tweets = list(dict.fromkeys(noRetweet+Retweet))
df = pd.DataFrame(tweets)

def vaderSentimentAnalysis(data_str):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(data_str)
    ss.pop('compound', None)
    maximum = max(ss, key=ss.get)  # Just use 'min' instead of 'max' for minimum.
    if maximum == 'neu':
        if (ss['neu'] >= 0.6):
            return 0
        elif (ss['pos'] > ss['neg']):
            return 1
        elif (ss['neg'] > ss['pos']):
            return 2
        else:
            return 0
    elif maximum == 'pos':
        return 1
    elif maximum == 'neg':
        return 2

def apply_vader(df):
    df['label'] = df[0].apply(vaderSentimentAnalysis)
    return df

def parallelize_dataframe(df, func, n_cores=2):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [2]:
%time df_labeled = parallelize_dataframe(df, apply_vader)

CPU times: user 761 ms, sys: 292 ms, total: 1.05 s
Wall time: 7min 4s
